In [ ]:
!pip install plotly

In [61]:
!pip uninstall -y numpy
!pip install numpy

Found existing installation: numpy 1.16.6
Uninstalling numpy-1.16.6:
  Successfully uninstalled numpy-1.16.6
     |████████████████████████████████| 15.7 MB 1.7 MB/s eta 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorlayer 2.1.0 requires numpy<1.17,>=1.16, but you have numpy 1.21.4 which is incompatible.
tensorflow 2.4.1 requires numpy~=1.19.2, but you have numpy 1.21.4 which is incompatible.
tensorflow 2.4.1 requires wrapt~=1.12.1, but you have wrapt 1.11.1 which is incompatible.


In [12]:
import numpy as np
import pandas as pd
import math
import time
from functools import partial

import plotly.express as px

import tensorflow as tf
import tensorflow.keras as keras

from sklearn.model_selection import train_test_split

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [22]:
np.random.seed(2021)
tf.random.set_seed(2021)

In [4]:
num_classes=10

x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

y_train_1hot = keras.utils.to_categorical(y_train, num_classes)
y_test_1hot = keras.utils.to_categorical(y_test, num_classes)

In [5]:
batch_size = 128
meta_batch_size = 32
epochs = 10
plt_x,plt_y=x_test[:batch_size],y_test[:batch_size]

In [6]:
train_loader = tf.data.Dataset.from_tensor_slices((x_train,y_train))
all_class_list=[train_loader.filter(lambda x,y: y==yy)
                  .cache()
                  .shuffle(2000)
                  .prefetch(50)
                 for yy in range(num_classes)]

In [7]:
WAYS=5
SHOTS=3
W,H,CH=28,28,1

In [8]:
def form_meta2(dataset_list,mult=4):
    orders=np.concatenate([np.random.permutation(len(dataset_list)) for x in range(mult)])
    for tasks in range(len(orders)//WAYS):
        #從已決定好的順序拉出WAY個class#
        picked=[dataset_list[tt] for tt in orders[WAYS*tasks:WAYS*(tasks+1)]]
        #每個class抽SHOTS+1張#
        data = [next(iter(class_loader.batch(SHOTS+1)))[0] for class_loader in picked]
        #support每個class各有SHOTS張照片#
        support=tf.concat([d[:-1] for d in data],axis=0)
        #query挑每個class 1 張，順序不固定#
        idxs=np.random.choice(range(WAYS), size=WAYS, replace=False)
        query=tf.concat([data[wayid][-2:-1] for wayid in idxs],axis=0)
        #輸出的時候把support跟query接在一起#
        yield tf.concat([support, query], axis=0), tf.stack([keras.utils.to_categorical(idx,num_classes=WAYS) for idx in idxs], axis=0)

In [9]:
MULT=40
train_meta_loader=tf.data.Dataset.from_generator(partial(form_meta2,all_class_list,MULT),
                                                 output_types=(tf.float32,tf.float32),
                                                output_shapes=((WAYS*(SHOTS+1),W,H,CH),(WAYS,WAYS)))

In [10]:
start_time=time.time()
for ld in all_class_list:
    for xx,yy in ld.batch(500):
        pass
print(f"{time.time()-start_time:.03f} sec")

16.080 sec


In [11]:
start_time=time.time()
for ld in all_class_list:
    for xx,yy in ld.batch(500):
        pass
print(f"{time.time()-start_time:.03f} sec")

2.694 sec


In [27]:
[1,2,3,4][3:4]

[4]

In [36]:
## Embedding ##
class Encoder(keras.Model):
    def __init__(self,n_layers=2,latent_features=2,hidden_featrues=64):
        super().__init__()
        self.latent_features=latent_features
        # First Layer
        self.conv_0=keras.layers.Conv2D(hidden_featrues, kernel_size=3, activation="relu",padding='same')
        self.dropout_0=keras.layers.Dropout(0.2)
        # Middle Layers
        self.convs=[keras.layers.Conv2D(hidden_featrues, kernel_size=3, activation="relu",padding='same') for _ in range(n_layers)]
        self.mxpools = [keras.layers.MaxPooling2D(pool_size=2) for _ in range(n_layers)]
        self.dropouts = [keras.layers.Dropout(0.2) for _ in range(n_layers)]
        # Integration Layer(Output Layer)
        WIDTH=2
        self.gap=keras.layers.AveragePooling2D(pool_size=WIDTH,)
        self.flatten=keras.layers.Flatten()
        self.dense=keras.layers.Dense(latent_features)
      
    def call(self,x):
        x=self.dropout_0(self.conv_0(x))
        for conv,mxp,drp in zip(self.convs,self.mxpools,self.dropouts):
            x=drp(mxp(conv(x)))
        x=self.gap(x)
        x=self.flatten(x)
        y=self.dense(x)
        return y

## Special Layers ##

class MatMul:
    def __init__(self,mode="paired"):
        self.mode=mode
        assert mode in ["paired","cross"]
    def __call__(self,q,s):
        if self.mode=="paired":
            return tf.reduce_sum(tf.multiply(q,s),axis=-1,keepdims=True)
        if self.mode=="cross":
            return tf.matmul(q,s,transpose_b=True)

class EuclideanDist:
    def __init__(self,mode="paired"):
        self.mode=mode
        assert mode in ["paired","cross"]
        
    def __call__(self,q,s):
        if self.mode=="paired":
            dist=tf.math.reduce_euclidean_norm(q-s,axis=-1,keepdims=True)
        if self.mode=="cross":
            dist=-tf.math.reduce_euclidean_norm(
                tf.stack([q[:,qq:qq+1]-s for qq in range(q.shape[1])],axis=1)
                ,axis=-1,keepdims=False)
            
        return dist
            
        


class CosSim:
    def __init__(self,mode="paired"):
        self.mode=mode
        assert mode in ["paired","cross"]
        self.matmul=MatMul(mode)
    def __call__(self,q,s):
        query_n=tf.math.l2_normalize(q, axis=-1)
        support_n=tf.math.l2_normalize(s, axis=-1)
        return self.matmul(query_n,support_n)
    
class CosineLayer(keras.layers.Layer):
    def __init__(self, out_features=10,mode="cross"):
        super().__init__()
        self.out_features = out_features
        self.cos_sim=CosSim(mode)
    def build(self, input_shape):
        self.w = self.add_weight(shape=(self.out_features,input_shape[-1]),
                               initializer='glorot_uniform',
                               trainable=True)
    def call(self, inputs):
        return self.cos_sim(inputs,self.w)

## Loss function ##
class ContrastiveLoss:
    def __init__(self,m=1,mode="dist"):
        
        self.mode=mode
        if self.mode=="dist":
            self.m=m
        if self.mode=="cos" or self.mode=="matmul":
            self.activation=keras.activations.sigmoid
            self.loss_fn=keras.losses.BinaryCrossentropy()
        assert mode in ["dist","cos","matmul"]
        self.__name__="loss"
    def __call__(self,y_true, y_pred):
        if self.mode=="dist":
            loss=tf.reduce_mean(y_true * tf.square(y_pred) +
                                 (1 - y_true)* tf.square(tf.maximum(self.m - y_pred, 0)),
                               axis=-1,keepdims=True)
        if self.mode=="cos" or self.mode=="matmul":
            loss=self.loss_fn(y_true,self.activation(y_pred))
        return loss


class FocalLoss:
    def __init__(self, gamma=3, eps=1e-10):
        self.gamma = gamma
        self.eps = tf.constant(eps)
        self.softmax =keras.layers.Softmax()
        self.cce=keras.losses.CategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
        self.__name__="loss"
    def __call__(self,y_true, y_pred): 
        logp = self.cce(y_true,self.softmax(y_pred+self.eps))
        p = tf.exp(-logp)
        loss = (tf.constant(1.) - p) ** self.gamma * logp
        return tf.reduce_mean(loss, axis=-1)
    
class AddMarginLoss():
    def __init__(self,s=5,m=0.3,gamma=3):
        self.s = s
        self.m = m
        self.lossfn=FocalLoss(gamma)
        self.__name__="loss"
    def __call__(self, y_true, y_pred):
        cosine=y_pred
        phi = cosine - self.m
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        arc = (y_true * phi) + ((1.0 - y_true) * cosine)  # you can use torch.where if your torch.__version__ is 0.4
        arc *= self.s
        return self.lossfn(y_true,arc)
    
class ArcMarginLoss():
    def __init__(self,s=32,m=0.3,easy_margin=False,gamma=3, eps=1e-6):
        self.s = s
        self.m = m
        self.eps=tf.constant(eps)# This is very, very, very important
        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m
        
        self.lossfn=FocalLoss(gamma)
        self.__name__="loss"
    def __call__(self, y_true, y_pred):
        cosine=y_pred
        
        sine = tf.sqrt(tf.constant(1.) - tf.pow(cosine, 2) + self.eps)
        
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where((cosine - self.th) > 0, phi, cosine - self.mm)
        
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        arc = (y_true * phi) + ((1.0 - y_true) * cosine)  # you can use torch.where if your torch.__version__ is 0.4
        arc *= self.s
        return self.lossfn(y_true,arc)
    
## Main Model ##
class ClassModel(keras.Model):
    def __init__(self,encoder,out_features=10,mode="dense"):
        super().__init__()
        assert mode in ["dense","cosmargin","arcmargin"]
        self.encoder=encoder
        self.mode=mode
        self.latent_features=encoder.latent_features
        
        if self.mode=="dense":
            self.output_layer=keras.layers.Dense(out_features,use_bias=False)
            self.loss_fn=FocalLoss(3)
            return
        
        self.output_layer=CosineLayer(out_features)
        if self.mode=="cosmargin":
            self.loss_fn=AddMarginLoss(s=5,m=0.3)
        if self.mode=="arcmargin":
            self.loss_fn=ArcMarginLoss(s=5,m=0.4,easy_margin=False)
        
    def call(self,x,training=False):
        latent=self.encoder(x)
        return self.output_layer(latent)

class SiameseModel(keras.Model):
    # Mode: dist= euclidean distance
    #       cos = cosine distance
    #       matmult= matric multiplication distance
    def __init__(self,encoder,mode="cos"):
        super().__init__()
        assert mode in ["dist","cos","matmul"]
        self.encoder=encoder
        self.mode=mode
        self.latent_features=encoder.latent_features
        if self.mode=="dist":
            self.matric_fn=EuclideanDist("paired")
        if self.mode=="cos":
            self.matric_fn=CosSim("paired")
        if self.mode=="matmul":
            self.matric_fn=MatMul("paired")
        self.loss_fn=ContrastiveLoss(1,mode=mode)
            
    def call(self,x,training=False):
        latent_s=self.encoder(x[:,0])
        latent_q=self.encoder(x[:,1])

        return self.matric_fn(latent_q,latent_s)     
  
    
class PrototypicalNetworks(keras.Model):
    # Mode: dist= euclidean distance
    #       cos = cosine distance
    #       matmult= matric multiplication distance
    def __init__(self,encoder,mode="cos"):
        super().__init__()
        assert mode in ["dist","cos","matmul","cosmargin","arcmargin"]
        self.encoder=encoder
        self.mode=mode
        self.latent_features=encoder.latent_features
        #Metric function#    
        if mode in ["cos","cosmargin","arcmargin"]:
            self.matric_fn=CosSim("cross")
        if self.mode=="dist":
            self.matric_fn=EuclideanDist("cross")
        if self.mode=="matmul":
            self.matric_fn=MatMul("cross")
        #Loss function, including output activation#
        if mode in ["dist","cos","matmul"]:
            self.loss_fn=FocalLoss(3)
        if self.mode=="cosmargin":
            self.loss_fn=AddMarginLoss(s=5,m=0.3)
        if self.mode=="arcmargin":
            self.loss_fn=ArcMarginLoss(s=5,m=0.4,easy_margin=False)  
            
    def call(self,x,training=False):
        # 計算query latent
        latent_q=tf.stack([self.encoder(x[:,WAYS*SHOTS+ii]) for ii in range(WAYS)],axis=1)        
        
        # 計算 prototypes
        latent_s=[self.encoder(x[:,ii]) for ii in range(WAYS*SHOTS)]
        latent_proto=tf.stack([tf.reduce_mean(tf.stack(latent_s[ww*SHOTS:(ww+1)*SHOTS],axis=-1),axis=-1)
                     for ww in range(WAYS)],axis=1)
        # 計算metric
        return self.matric_fn(latent_q,latent_proto)

In [60]:
enc=Encoder(n_layers=2,latent_features=2,hidden_featrues=64)
model=PrototypicalNetworks(enc,mode="dist")# {"dist","cos","matmul","cosmargin","arcmargin"}

In [61]:
# loss_fn=ArcMarginLoss(s=5,m=0.4,easy_margin=False)
loss_fn=model.loss_fn
opt=keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss=loss_fn, optimizer=opt, metrics=[keras.metrics.CategoricalAccuracy()])

In [62]:
## Recording Callbacks
class LogLatent(keras.callbacks.Callback):
    def __init__(self,encoder,test_x,test_y,skip_iters=100):
        self.df=self.df=pd.DataFrame()
        self.encoder=encoder
        self.plt_x=test_x
        self.plt_y=test_y
        self.itr=0
        self.skip_iters=skip_iters
    def on_batch_end(self,batch, logs={}):
        if self.itr%self.skip_iters==self.skip_iters-1:
            latent=self.encoder(self.plt_x)
            df_new=pd.DataFrame()
            df_new["f1"]=latent[:,0]
            df_new["f2"]=latent[:,1]
            df_new["arc"]=0.5
            df_new["iterations"]=np.repeat(self.itr,len(latent))

            df_new["label"]=self.plt_y
            self.df=self.df.append(df_new)
        self.itr+=1
    def normalize(self):
        # Normalize latent by maximum value
        self.mx_norm=np.linalg.norm(self.df[["f1","f2"]].values,axis=1).max()
        self.df[["f1",'f2']]=self.df[["f1",'f2']].apply(lambda x: x/self.mx_norm)

        # Project latents into arc
        df=self.df.copy()
        norm=np.linalg.norm(df[["f1","f2"]].values,axis=1,keepdims=True)
        df[["f1",'f2']]=df[["f1",'f2']].values/norm
        df["arc"]=df["arc"].map({0.5:0.9},na_action=None)
        
        self.df=self.df.append(df)

In [41]:

train_loader = train_meta_loader.shuffle(1000).cache().prefetch(meta_batch_size).batch(meta_batch_size)

val_loader = train_meta_loader.shuffle(1000).cache().prefetch(meta_batch_size*4).batch(meta_batch_size*4)


In [63]:
lg=LogLatent(enc,plt_x,plt_y)
try:
    model.fit(train_loader, 
              epochs=800,
              validation_data=val_loader,
              callbacks=[lg])
except KeyboardInterrupt:
    print("KeyboardInterrupt")

Epoch 1/800
3/3 [==============================] - 6s 517ms/step - loss: 0.8116 - categorical_accuracy: 0.3478 - val_loss: 0.7703 - val_categorical_accuracy: 0.4975
Epoch 2/800
3/3 [==============================] - 0s 71ms/step - loss: 0.7422 - categorical_accuracy: 0.5141 - val_loss: 0.6809 - val_categorical_accuracy: 0.5200
Epoch 3/800
3/3 [==============================] - 0s 64ms/step - loss: 0.6417 - categorical_accuracy: 0.4905 - val_loss: 0.5575 - val_categorical_accuracy: 0.5350
Epoch 4/800
3/3 [==============================] - 0s 64ms/step - loss: 0.5154 - categorical_accuracy: 0.5375 - val_loss: 0.4716 - val_categorical_accuracy: 0.5525
Epoch 5/800
3/3 [==============================] - 0s 71ms/step - loss: 0.4673 - categorical_accuracy: 0.5528 - val_loss: 0.4351 - val_categorical_accuracy: 0.5825
Epoch 6/800
3/3 [==============================] - 0s 65ms/step - loss: 0.4388 - categorical_accuracy: 0.5497 - val_loss: 0.4008 - val_categorical_accuracy: 0.5875
Epoch 7/800
3/3

Epoch 51/800
3/3 [==============================] - 0s 66ms/step - loss: 0.0501 - categorical_accuracy: 0.9014 - val_loss: 0.1428 - val_categorical_accuracy: 0.8100
Epoch 52/800
3/3 [==============================] - 0s 64ms/step - loss: 0.0600 - categorical_accuracy: 0.9108 - val_loss: 0.1369 - val_categorical_accuracy: 0.8100
Epoch 53/800
3/3 [==============================] - 0s 65ms/step - loss: 0.0562 - categorical_accuracy: 0.8939 - val_loss: 0.1365 - val_categorical_accuracy: 0.8150
Epoch 54/800
3/3 [==============================] - 0s 65ms/step - loss: 0.0561 - categorical_accuracy: 0.9197 - val_loss: 0.1395 - val_categorical_accuracy: 0.8150
Epoch 55/800
3/3 [==============================] - 0s 64ms/step - loss: 0.0550 - categorical_accuracy: 0.9206 - val_loss: 0.1470 - val_categorical_accuracy: 0.8000
Epoch 56/800
3/3 [==============================] - 0s 65ms/step - loss: 0.0531 - categorical_accuracy: 0.9066 - val_loss: 0.1543 - val_categorical_accuracy: 0.7800
Epoch 57/8

Epoch 101/800
3/3 [==============================] - 0s 67ms/step - loss: 0.0345 - categorical_accuracy: 0.9341 - val_loss: 0.1767 - val_categorical_accuracy: 0.8100
Epoch 102/800
3/3 [==============================] - 0s 67ms/step - loss: 0.0240 - categorical_accuracy: 0.9537 - val_loss: 0.1667 - val_categorical_accuracy: 0.8100
Epoch 103/800
3/3 [==============================] - 0s 66ms/step - loss: 0.0298 - categorical_accuracy: 0.9500 - val_loss: 0.1541 - val_categorical_accuracy: 0.8125
Epoch 104/800
3/3 [==============================] - 0s 64ms/step - loss: 0.0198 - categorical_accuracy: 0.9527 - val_loss: 0.1492 - val_categorical_accuracy: 0.8050
Epoch 105/800
3/3 [==============================] - 0s 66ms/step - loss: 0.0228 - categorical_accuracy: 0.9630 - val_loss: 0.1507 - val_categorical_accuracy: 0.8100
Epoch 106/800
3/3 [==============================] - 0s 65ms/step - loss: 0.0226 - categorical_accuracy: 0.9470 - val_loss: 0.1572 - val_categorical_accuracy: 0.8050
Epoc

3/3 [==============================] - 0s 65ms/step - loss: 0.0086 - categorical_accuracy: 0.9803 - val_loss: 0.1897 - val_categorical_accuracy: 0.8100
Epoch 151/800
3/3 [==============================] - 0s 67ms/step - loss: 0.0119 - categorical_accuracy: 0.9795 - val_loss: 0.1887 - val_categorical_accuracy: 0.8150
Epoch 152/800
3/3 [==============================] - 0s 68ms/step - loss: 0.0123 - categorical_accuracy: 0.9739 - val_loss: 0.1908 - val_categorical_accuracy: 0.8150
Epoch 153/800
3/3 [==============================] - 0s 65ms/step - loss: 0.0195 - categorical_accuracy: 0.9692 - val_loss: 0.1918 - val_categorical_accuracy: 0.8100
Epoch 154/800
3/3 [==============================] - 0s 64ms/step - loss: 0.0145 - categorical_accuracy: 0.9639 - val_loss: 0.1927 - val_categorical_accuracy: 0.8125
Epoch 155/800
3/3 [==============================] - 0s 64ms/step - loss: 0.0160 - categorical_accuracy: 0.9616 - val_loss: 0.1872 - val_categorical_accuracy: 0.8100
Epoch 156/800
3/3 

3/3 [==============================] - 0s 64ms/step - loss: 0.0096 - categorical_accuracy: 0.9728 - val_loss: 0.2017 - val_categorical_accuracy: 0.8225
Epoch 200/800
3/3 [==============================] - 0s 70ms/step - loss: 0.0077 - categorical_accuracy: 0.9791 - val_loss: 0.2132 - val_categorical_accuracy: 0.8150
Epoch 201/800
3/3 [==============================] - 0s 66ms/step - loss: 0.0041 - categorical_accuracy: 0.9919 - val_loss: 0.2314 - val_categorical_accuracy: 0.8050
Epoch 202/800
3/3 [==============================] - 0s 64ms/step - loss: 0.0141 - categorical_accuracy: 0.9719 - val_loss: 0.2364 - val_categorical_accuracy: 0.7975
Epoch 203/800
3/3 [==============================] - 0s 65ms/step - loss: 0.0102 - categorical_accuracy: 0.9683 - val_loss: 0.2238 - val_categorical_accuracy: 0.8000
Epoch 204/800
3/3 [==============================] - 0s 65ms/step - loss: 0.0081 - categorical_accuracy: 0.9816 - val_loss: 0.2080 - val_categorical_accuracy: 0.8050
Epoch 205/800
3/3 

...

3/3 [==============================] - 0s 66ms/step - loss: 0.0019 - categorical_accuracy: 0.9980 - val_loss: 0.2355 - val_categorical_accuracy: 0.8225
Epoch 592/800
3/3 [==============================] - 0s 67ms/step - loss: 0.0026 - categorical_accuracy: 0.9944 - val_loss: 0.2494 - val_categorical_accuracy: 0.8225
Epoch 593/800
3/3 [==============================] - 0s 66ms/step - loss: 3.5894e-04 - categorical_accuracy: 1.0000 - val_loss: 0.2609 - val_categorical_accuracy: 0.8225
Epoch 594/800
3/3 [==============================] - 0s 65ms/step - loss: 0.0045 - categorical_accuracy: 0.9880 - val_loss: 0.2617 - val_categorical_accuracy: 0.8250
Epoch 595/800
3/3 [==============================] - 0s 64ms/step - loss: 0.0039 - categorical_accuracy: 0.9875 - val_loss: 0.2536 - val_categorical_accuracy: 0.8275
Epoch 596/800
3/3 [==============================] - 0s 68ms/step - loss: 0.0014 - categorical_accuracy: 0.9964 - val_loss: 0.2455 - val_categorical_accuracy: 0.8225
Epoch 597/800


3/3 [==============================] - 0s 68ms/step - loss: 0.0022 - categorical_accuracy: 0.9944 - val_loss: 0.2387 - val_categorical_accuracy: 0.8100
Epoch 641/800
3/3 [==============================] - 0s 68ms/step - loss: 6.5889e-04 - categorical_accuracy: 0.9988 - val_loss: 0.2384 - val_categorical_accuracy: 0.8175
Epoch 642/800
3/3 [==============================] - 0s 67ms/step - loss: 0.0015 - categorical_accuracy: 0.9964 - val_loss: 0.2391 - val_categorical_accuracy: 0.8175
Epoch 643/800
3/3 [==============================] - 0s 69ms/step - loss: 3.9734e-04 - categorical_accuracy: 1.0000 - val_loss: 0.2407 - val_categorical_accuracy: 0.8125
Epoch 644/800
3/3 [==============================] - 0s 67ms/step - loss: 8.5578e-04 - categorical_accuracy: 1.0000 - val_loss: 0.2463 - val_categorical_accuracy: 0.8050
Epoch 645/800
3/3 [==============================] - 0s 66ms/step - loss: 3.4369e-04 - categorical_accuracy: 1.0000 - val_loss: 0.2539 - val_categorical_accuracy: 0.8075
Ep

Epoch 689/800
3/3 [==============================] - 0s 70ms/step - loss: 5.5459e-04 - categorical_accuracy: 1.0000 - val_loss: 0.2502 - val_categorical_accuracy: 0.8150
Epoch 690/800
3/3 [==============================] - 0s 67ms/step - loss: 7.8286e-04 - categorical_accuracy: 0.9964 - val_loss: 0.2525 - val_categorical_accuracy: 0.8125
Epoch 691/800
3/3 [==============================] - 0s 71ms/step - loss: 2.7508e-04 - categorical_accuracy: 1.0000 - val_loss: 0.2555 - val_categorical_accuracy: 0.8125
Epoch 692/800
3/3 [==============================] - 0s 68ms/step - loss: 0.0016 - categorical_accuracy: 0.9944 - val_loss: 0.2550 - val_categorical_accuracy: 0.8150
Epoch 693/800
3/3 [==============================] - 0s 66ms/step - loss: 0.0033 - categorical_accuracy: 0.9964 - val_loss: 0.2643 - val_categorical_accuracy: 0.8125
Epoch 694/800
3/3 [==============================] - 0s 70ms/step - loss: 2.7035e-04 - categorical_accuracy: 1.0000 - val_loss: 0.2718 - val_categorical_accur

3/3 [==============================] - 0s 66ms/step - loss: 0.0014 - categorical_accuracy: 0.9988 - val_loss: 0.2687 - val_categorical_accuracy: 0.8200
Epoch 738/800
3/3 [==============================] - 0s 67ms/step - loss: 0.0021 - categorical_accuracy: 0.9892 - val_loss: 0.2487 - val_categorical_accuracy: 0.8300
Epoch 739/800
3/3 [==============================] - 0s 66ms/step - loss: 6.9695e-04 - categorical_accuracy: 0.9988 - val_loss: 0.2386 - val_categorical_accuracy: 0.8300
Epoch 740/800
3/3 [==============================] - 0s 64ms/step - loss: 0.0010 - categorical_accuracy: 0.9980 - val_loss: 0.2364 - val_categorical_accuracy: 0.8275
Epoch 741/800
3/3 [==============================] - 0s 65ms/step - loss: 0.0031 - categorical_accuracy: 0.9980 - val_loss: 0.2367 - val_categorical_accuracy: 0.8225
Epoch 742/800
3/3 [==============================] - 0s 66ms/step - loss: 2.1149e-04 - categorical_accuracy: 1.0000 - val_loss: 0.2373 - val_categorical_accuracy: 0.8250
Epoch 743/

Epoch 786/800
3/3 [==============================] - 0s 66ms/step - loss: 0.0069 - categorical_accuracy: 0.9944 - val_loss: 0.2729 - val_categorical_accuracy: 0.8125
Epoch 787/800
3/3 [==============================] - 0s 65ms/step - loss: 8.4403e-04 - categorical_accuracy: 0.9980 - val_loss: 0.2670 - val_categorical_accuracy: 0.8125
Epoch 788/800
3/3 [==============================] - 0s 65ms/step - loss: 6.4908e-04 - categorical_accuracy: 1.0000 - val_loss: 0.2674 - val_categorical_accuracy: 0.8150
Epoch 789/800
3/3 [==============================] - 0s 66ms/step - loss: 0.0024 - categorical_accuracy: 0.9939 - val_loss: 0.2676 - val_categorical_accuracy: 0.8200
Epoch 790/800
3/3 [==============================] - 0s 66ms/step - loss: 4.7929e-04 - categorical_accuracy: 1.0000 - val_loss: 0.2685 - val_categorical_accuracy: 0.8200
Epoch 791/800
3/3 [==============================] - 0s 65ms/step - loss: 0.0013 - categorical_accuracy: 0.9964 - val_loss: 0.2693 - val_categorical_accuracy:

In [64]:
lg.normalize()

In [ ]:
fig=px.scatter(lg.df, x="f1", y="f2", animation_frame="iterations", color="label",opacity=lg.df.arc,
                range_x=[-1.5,1.5],
              range_y=[-1.5,1.5])
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
)

In [66]:
fig.write_html("history/proto_euc_dist_tf.html")